In [10]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
from collections import defaultdict

In [13]:
class Translation_Api():
    # https://docs.microsoft.com/en-us/azure/cognitive-services/translator/reference/v3-0-reference
    def __init__(self):
        self.headers = {
                "Ocp-Apim-Subscription-Key":"e669322a133044489e6dc4e9cde3edee",
                "Content-Type":"application/json",
                "Ocp-Apim-Subscription-Region":"centralus"
            }
        self.transliterate_url = "https://api.cognitive.microsofttranslator.com/transliterate?api-version=3.0&language=hi&fromScript=Latn&toScript=Deva"
        self.translate_url = "https://api.cognitive.microsofttranslator.com/translate?api-version=3.0&to=hi&from=en&toScript=Deva"
    
    def get_translation(self,data):
        if "{{" in data : return data
        data = data.split("<ref>")[0]
        if data == "" : return data
        data = [{"Text":data}]
        res = r.post( self.translate_url , json = data , headers = self.headers).text
        res = json.loads(res)
        res = [ ret['translations'][0]['text'] for ret in res ]
        return res[0].replace("[[","[").replace("]]","]").replace("[","[[").replace("]","]]")
        
    def get_transliteration(self,data):
        if "{{" in data : return data
        data = data.split("<ref>")[0]
        if data == "" : return data
        data = [{"Text":data}]
        res = r.post( self.transliterate_url , json = data , headers = self.headers).text
        res = json.loads(res)
        res = [ret['text'] for ret in res]
        return res[0].replace("[[","[").replace("]]","]").replace("[","[[").replace("]","]]")
translator = Translation_Api()

### Program

In [18]:
biography = {
    "image": "P18",
    "gender":"P21",
    "residence":"P551",
    "birth_place":"P19",
    "birth_date":"P569",
    "profession": "P106",
    "notable_works": "P800",
    "education": "P69",
    "positions":"P39",
    "awards": "P166",
    "spouse": "P26",
    "nationality": "P27",
}

translation = {
    "name":"नाम",
    "description":"विवरण",
    "image": "चित्र",
    "gender":"लिंग",
    "residence":"निवास",
    "birth_place":"जन्म स्थान",
    "birth_date":"जन्मतारीख",
    "profession": "व्यवसाय",
    "notable_works": "उल्लेखनीय कार्य",
    "education": "शिक्षा",
    "positions": "पद",
    "awards": "पुरस्कार",
    "spouse": "पति या पत्नी",
    "other_available_information":"अन्य उपलब्ध जानकारी",
    "main_info": "मुख्य जानकारी",
    "nationality": "राष्ट्रीयता"
}

to_transliteration = ['name','birth_place','spouse','notable_works']

In [19]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

endpoint_url = "https://query.wikidata.org/sparql"

In [30]:
#calls qwikidata get entity function for an entity id
def getEntityInfo(eid):
    return get_entity_dict_from_api(eid)

#extract the name of the entity or the property value in native language
def extractName(info):
    return info.get('labels', {}).get('en', {}).get('value', "")

#extract the description of the entity or the property value in native language
def extractDescription(info):
    return info.get('descriptions', {}).get('en', {}).get('value', "")

#print the name and the description
def printNameAndDescription(info, trans):
    name = extractName(info)
    if name != "":
        print(trans['name'] + ":", translator.get_transliteration(name))
    desc = extractDescription(info)
    if desc != "":
        print(trans['description'] + ":", translator.get_translation(desc))
        
#print the information present in the entity itself
def printOtherInfo(entity, bio):
    #get the claims subdict
#     print(entity['claims'])
    for p in entity['claims'].keys():
        if p not in bio.values():
            #get information on the property in bengali
            ent_info = getEntityInfo(p)
            name, desc = extractName(ent_info), extractDescription(ent_info)
            if name == "":
                continue
            value = ""
            #for every property in the claims subdict get inforamtion on the correspoding values
            for data in entity.get('claims', {}).get(p, []):
                res = data.get('mainsnak',{}).get('datavalue', {}).get('value', {})
                if type(res) == dict:
                    info_id = res.get('id', "")
                    if info_id == "":
                        continue
                    info = getEntityInfo(info_id)
                    pname, pdesc = extractName(info), extractDescription(info)
                    if pname == "":
                        continue
                    if pdesc != "":
                        pname += str(f'({pdesc})')
                    value += pname 
            #only print property name and value if the value is present in native language
            if value != "":
                print(name + ": " + value)
    
def getInfobox(wd, bio, trans):
    #get entity from api
    entity_info = getEntityInfo(wd)
    #print name and description
    printNameAndDescription(entity_info, trans)
    #explicitly query using sparql to get main biography data
    print("----------------------",trans['main_info'],"----------------------")
    for entity, wdt in bio.items():
        spqrqlq = f"SELECT ?entity ?entityLabel ?entityDescription WHERE {{ wd:{wd} wdt:{wdt} ?entity; SERVICE wikibase:label {{ bd:serviceParam wikibase:language \"en\". }} }}"
        s, v = trans[entity] + ": ", ""
        res = get_results(endpoint_url, str(spqrqlq))
        for entity in res['results']['bindings']:
            value = entity.get('entityLabel').get('value', "")
            if value != '' and 'Q' not in value:
                v += value + ','
        if v != "":
            if entity in to_transliteration:
                v = translator.get_transliteration(v)
            else:
                v = translator.get_translation(v)
            print(s + v)
    #Query and print the rest of the data already present in the entity 
#     print("----------------------",trans['other_available_information'],"----------------------")
#     printOtherInfo(entity_info, bio)

In [32]:
# Information about Shane Warne
getInfobox('Q555240', biography, translation)

नाम: शेन वार्न
विवरण: ऑस्ट्रेलियाई पूर्व अंतरराष्ट्रीय क्रिकेटर
---------------------- मुख्य जानकारी ----------------------
चित्र: http://commons.wikimedia.org/wiki/Special:FilePath/Shane%20Warne%202011%20cropped.jpg,
लिंग: पुरुष
जन्म स्थान: ऊपरी फर्नट्री गली,
जन्मतारीख: 1969-09-13T00:00:00Z,
व्यवसाय: क्रिकेटर, पोकर खिलाड़ी,
शिक्षा: हैम्पटन हाई स्कूल, मेलबोर्न, मेंटोन व्याकरण स्कूल,
पुरस्कार: विस्डेन क्रिकेटर ऑफ द ईयर,
राष्ट्रीयता: ऑस्ट्रेलिया


In [31]:
# Information about Narendra Modi
getInfobox('Q1058', biography, translation)

नाम: नरेन्द्र मोदी
विवरण: भारत के प्रधानमंत्री
---------------------- मुख्य जानकारी ----------------------
चित्र: http://commons.wikimedia.org/wiki/Special:FilePath/PM%20Modi%202015.jpg,
लिंग: पुरुष
निवास: 7, लोक कल्याण मार्ग,
जन्म स्थान: वडनगर,
जन्मतारीख: 1950-09-17T00:00:00Z,
व्यवसाय: नेता
शिक्षा: गुजरात विश्वविद्यालय, दिल्ली विश्वविद्यालय,
पद: भारत की एक विधान सभा के सदस्य, कार्मिक, लोक शिकायत और पेंशन मंत्रालय, गुजरात विधानसभा के सदस्य, गुजरात के मुख्यमंत्री, 16वीं लोकसभा के सदस्य, 17वीं लोकसभा के सदस्य,
पुरस्कार: अब्दुलअजीज अल सऊद का आदेश, सेंट एंड्रयू का आदेश, जायद का आदेश, सीएनएन-न्यूज18 इंडियन ऑफ द ईयर, अमीर अमानुल्लाह खान पुरस्कार, फिलिस्तीन राज्य के ग्रैंड कॉलर, निशन इज्जुद्दीन के प्रतिष्ठित शासन का आदेश,
पति या पत्नी: जशोदाबेन नरेंद्रभाई मोदी,
राष्ट्रीयता: भारत
